* 설치

In [ ]:
%%capture --no-stderr
%pip install python-dotenv langchain langchain-openai

* 실행

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

#llm = ChatOpenAI(model="gpt-4o")
llm = ChatOpenAI(model=os.getenv("OPENAI_API_MODEL"))

In [ ]:
# llm 다이렉트로 호출
result = llm.invoke("2 # 3 값은 몇이야?")
print(result)

In [ ]:
from langchain_core.tools import tool
from pprint import pprint

@tool
def operator_sharp(a: int, b: int) -> int:
    """calcurate a # b.

    Args:
        a: first int
        b: second int
    """
    #print(f"call by llm operator # ({a}, {b})")
    return a * b

#result = operator_charp({"a":10, "b":20})
#print(result)

llm_with_tools = llm.bind_tools([operator_sharp])
result = llm_with_tools.invoke("2 # 3 값은 몇이야?")
#print(result)
pprint(result.additional_kwargs)

In [ ]:
# chain으로 호출
from operator import attrgetter
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}")
])

chain = prompt | llm_with_tools | attrgetter("tool_calls") | operator_sharp.map()
result = chain.invoke("2 # 3하면 몇이야?")
print(result)
